In [1]:
import neutron_analysis.shielding as shielding
import neutron_analysis.utils as utils

import pandas as pd
import numpy as np

from io import StringIO

# Shielding

In [86]:
# remote copy of data
file = 'https://github.com/wpk-nist-gov/neutron_analysis/blob/develop/notebooks/example_input_data.xlsx?raw=true'
# local copy of data
#file = './example_input_data.xlsx'

In [87]:
df_mass = (
    shielding.read_mass_excel(file, sheet_name='Sheet1', extra_index=['sample','mass'])
)
df_mass.head()

mass_percent  mass_frac
sample   mass radius length density  element                         
sample_0 0.3  0.65   0.15   1.506792 H                0.75     0.0075
                                     C                2.00     0.0200
                                     N                1.00     0.0100
                                     O               47.00     0.4700
                                     Na               1.22     0.0122

In [88]:
# create shielding objects
# This object uses the user supplied table of element, sigma_a, sigma_s, atomic_mass
S = shielding.NeutronShieldingUser(df_mass)

In [89]:
# the input mass table
S.mass_table.head()

mass_percent  mass_frac
sample   mass radius length density  element                         
sample_0 0.3  0.65   0.15   1.506792 H                0.75     0.0075
                                     C                2.00     0.0200
                                     N                1.00     0.0100
                                     O               47.00     0.4700
                                     Na               1.22     0.0122

In [90]:
# the merged sigma_{a,s}, atomic mass table
S.sigma_table.head()

mass_percent  mass_frac  \
sample   mass radius length density  element                            
sample_0 0.3  0.65   0.15   1.506792 H                0.75     0.0075   
                                     C                2.00     0.0200   
                                     N                1.00     0.0100   
                                     O               47.00     0.4700   
                                     Na               1.22     0.0122   

                                              atomic_mass  sigma_a  sigma_s  \
sample   mass radius length density  element                                  
sample_0 0.3  0.65   0.15   1.506792 H           1.007940  0.33260   82.020   
                                     C          12.011000  0.00350    5.551   
                                     N          14.006740  1.90000   11.510   
                                     O          15.999400  0.00019    4.232   
                                     Na         22.989768  0.53000    3.280   

                                               Sigma_a   Sigma_s  
sample   mass radius length density  element                      
sample_0 0.3  0.65   0.15   1.506792 H        0.002246  0.553784  
                                     C        0.000005  0.008387  
                                     N        0.001231  0.007456  
                                     O        0.000005  0.112806  
                                     Na       0.000255  0.001579

In [91]:
# the summed table
S.sigma_table_tot.head()

,,,,,mass_frac,Sigma_a,Sigma_s,Sigma_t
sample,mass,radius,length,density,,,,
sample_0,0.3,0.65,0.15,1.506792,0.99876,0.010283,0.718763,0.729046
sample_1,0.4,0.65,0.15,2.009056,0.99876,0.013711,0.958350,0.972061
sample_2,0.3,0.65,0.15,1.506792,1.00126,0.011032,0.903357,0.914389


In [92]:
# shielding calculations
S.output_scatter

,,,,,mass_frac,Sigma_a,Sigma_s,Sigma_t,x,f_slab,f_cyl,f0,f
sample,mass,radius,length,density,,,,,,,,,
sample_0,0.3,0.65,0.15,1.506792,0.99876,0.010283,0.718763,0.729046,0.088852,0.850142,0.908769,0.861135,0.997731
sample_1,0.4,0.65,0.15,2.009056,0.99876,0.013711,0.958350,0.972061,0.118470,0.816646,0.886428,0.829730,0.997114
sample_2,0.3,0.65,0.15,1.506792,1.00126,0.011032,0.903357,0.914389,0.111441,0.824247,0.891449,0.836847,0.997653


In [93]:
S.output_noscatter

,,,,,mass_frac,Sigma_a,Sigma_s,Sigma_t,x,f_slab,f_cyl,f0,f
sample,mass,radius,length,density,,,,,,,,,
sample_0,0.3,0.65,0.15,1.506792,0.99876,0.010283,0.718763,0.729046,0.001253,0.995234,0.998341,0.995817,1.407294
sample_1,0.4,0.65,0.15,2.009056,0.99876,0.013711,0.958350,0.972061,0.001671,0.993886,0.997793,0.994619,1.594318
sample_2,0.3,0.65,0.15,1.506792,1.00126,0.011032,0.903357,0.914389,0.001344,0.994935,0.998221,0.995551,1.566136


In [94]:
# shielding table
S.shielding_table

,,,,,f_scatter,f0_noscatter
sample,mass,radius,length,density,,
sample_0,0.3,0.65,0.15,1.506792,0.997731,0.995817
sample_1,0.4,0.65,0.15,2.009056,0.997114,0.994619
sample_2,0.3,0.65,0.15,1.506792,0.997653,0.995551


In [95]:
# ratio of standard to other values and uncertainty
# std_val is the name of the sample that is the std (default="std")
S.shielding_ratio(std_val='sample_0')

sample_0/sample                 uncert  \
                                           f_scatter f0_noscatter f_scatter   
sample   mass radius length density                                           
sample_0 0.3  0.65   0.15   1.506792        1.000000     1.000000  0.000000   
sample_1 0.4  0.65   0.15   2.009056        1.000619     1.001205  0.000126   
sample_2 0.3  0.65   0.15   1.506792        1.000077     1.000267  0.000016   

                                                   
                                     f0_noscatter  
sample   mass radius length density                
sample_0 0.3  0.65   0.15   1.506792     0.000000  
sample_1 0.4  0.65   0.15   2.009056     0.000245  
sample_2 0.3  0.65   0.15   1.506792     0.000054

In [96]:
# to include "values", supply top level name
S.shielding_ratio('sample_0', value_name='values')

values              sample_0/sample  \
                                     f_scatter f0_noscatter       f_scatter   
sample   mass radius length density                                           
sample_0 0.3  0.65   0.15   1.506792  0.997731     0.995817        1.000000   
sample_1 0.4  0.65   0.15   2.009056  0.997114     0.994619        1.000619   
sample_2 0.3  0.65   0.15   1.506792  0.997653     0.995551        1.000077   

                                                     uncert               
                                     f0_noscatter f_scatter f0_noscatter  
sample   mass radius length density                                       
sample_0 0.3  0.65   0.15   1.506792     1.000000  0.000000     0.000000  
sample_1 0.4  0.65   0.15   2.009056     1.001205  0.000126     0.000245  
sample_2 0.3  0.65   0.15   1.506792     1.000267  0.000016     0.000054

## Using periodictable package

the python package `periodictable` provides atomic masses, $\sigma_a$ and $\sigma_s$ values

If you'd prefer to use this package for these properties, there is an alternative class shielding.NeutronShieldingPT


There are some small differences between `periodictable` parameters and those supplied in the user table.  See last section of this notebook for examples of differences

In [101]:
S0 = shielding.NeutronShieldingPT(df_mass)

In [102]:
S.shielding_ratio('sample_0')

sample_0/sample                 uncert  \
                                           f_scatter f0_noscatter f_scatter   
sample   mass radius length density                                           
sample_0 0.3  0.65   0.15   1.506792        1.000000     1.000000  0.000000   
sample_1 0.4  0.65   0.15   2.009056        1.000619     1.001205  0.000126   
sample_2 0.3  0.65   0.15   1.506792        1.000077     1.000267  0.000016   

                                                   
                                     f0_noscatter  
sample   mass radius length density                
sample_0 0.3  0.65   0.15   1.506792     0.000000  
sample_1 0.4  0.65   0.15   2.009056     0.000245  
sample_2 0.3  0.65   0.15   1.506792     0.000054

In [103]:
S0.shielding_ratio('sample_0')

sample_0/sample                 uncert  \
                                           f_scatter f0_noscatter f_scatter   
sample   mass radius length density                                           
sample_0 0.3  0.65   0.15   1.506792        1.000000     1.000000  0.000000   
sample_1 0.4  0.65   0.15   2.009056        1.000619     1.001205  0.000126   
sample_2 0.3  0.65   0.15   1.506792        1.000077     1.000267  0.000016   

                                                   
                                     f0_noscatter  
sample   mass radius length density                
sample_0 0.3  0.65   0.15   1.506792     0.000000  
sample_1 0.4  0.65   0.15   2.009056     0.000245  
sample_2 0.3  0.65   0.15   1.506792     0.000054

# photon attenuation

In [18]:
import neutron_analysis.photonatten as photonatten
import pandas as pd

In [104]:
df_mass = (
    shielding.read_mass_excel(file, sheet_name='mass', extra_index=['sample','mass'])
    
)

df_gamma = (
    pd.read_excel(file, 'gamma')
    .rename(columns={'element': 'element_tot'})
    .assign(element=lambda x: x['element_tot'].str.split('-').str.get(0))
    # energy should be in MeV
    # convert kev to mev
    .assign(energy=lambda x: x['gamma_kev'] / 1000.)
    .set_index('element')

)


S = shielding.NeutronShieldingUser(df_mass)
p = photonatten.PhotonAtten(df_gamma, df_mass)


In [105]:
# table of energies
# must have 'element' in index, an 'energy' in columns
p.gamma_table

,element_tot,gamma_kev,gamma_mev_used,energy
element,,,,
Mg,Mg-27,843.00,0.84300,0.84300
Al,Al-28,1778.80,1.77800,1.77880
Ca,Ca-49,3084.50,3.08400,3.08450
Cl,Cl-38,1642.59,1.64300,1.64259
Mn,Mn-56,1810.70,1.81700,1.81070
Na,Na-24,1368.55,1.36855,1.36855
Ti,Ti-51,320.00,0.32008,0.32000
V,V-52,1434.00,1.43400,1.43400
Sc,Sc-46,889.60,0.88960,0.88960


In [106]:
# mass table 
# must of 'radius','length','density','element' in index and 'mass_frac' in columns
p.mass_table.head()

mass_frac track
sample mass     radius length density  element                 
std    0.107074 0.65   0.15   0.537795 As        0.000043   NaN
                                       Cd        0.000049   NaN
                                       W         0.000035   NaN
                                       Au        0.000009   NaN
                                       Sm        0.000007   NaN

In [107]:
# atten table scraped from https://physics.nist.gov/PhysRefData/XrayMassCoef/tab3.html 
photonatten.get_default_atten_table().query('element=="Al"').head()

,energy,mu_rho,muen_rho,element
438,0.00100,1185.0,1183.0,Al
439,0.00150,402.2,400.1,Al
440,0.00156,362.1,360.0,Al
441,0.00156,3957.0,3829.0,Al
442,0.00200,2263.0,2204.0,Al


In [108]:
# atten_table: interpolate table above for each element in p.mass_table and energy in p.gamma_table
# Note that in the final table, element corresponds to elements in p.gamma_table, and element_other are elements in 
# p.mass_table
p.atten_table

mass_frac  \
sample mass     radius length density  element energy element_other              
std    0.107074 0.65   0.15   0.537795 Mg      0.8430 As              0.000043   
                                                      Au              0.000009   
                                                      C               0.444153   
                                                      Cd              0.000049   
                                                      H               0.062117   
...                                                                        ...   
QC     0.300000 0.65   0.15   1.506792 Cs      0.7962 Na              0.012200   
                                                      O               0.470000   
                                                      P               0.000688   
                                                      Si              0.303000   
                                                      Ti              0.003360   

                                                                       mu_rho  \
sample mass     radius length density  element energy element_other             
std    0.107074 0.65   0.15   0.537795 Mg      0.8430 As             0.062884   
                                                      Au             0.082483   
                                                      C              0.069223   
                                                      Cd             0.064885   
                                                      H              0.137447   
...                                                                       ...   
QC     0.300000 0.65   0.15   1.506792 Cs      0.7962 Na             0.068060   
                                                      O              0.071057   
                                                      P              0.069024   
                                                      Si             0.071009   
                                                      Ti             0.065902   

                                                                     muen_rho  
sample mass     radius length density  element energy element_other            
std    0.107074 0.65   0.15   0.537795 Mg      0.8430 As             0.025992  
                                                      Au             0.042331  
                                                      C              0.028650  
                                                      Cd             0.027987  
                                                      H              0.056997  
...                                                                       ...  
QC     0.300000 0.65   0.15   1.506792 Cs      0.7962 Na             0.027613  
                                                      O              0.028883  
                                                      P              0.027944  
                                                      Si             0.028764  
                                                      Ti             0.026628  

[640 rows x 3 columns]

In [109]:
# total attenuation calculated by summing over 'element_other' in p.atten_table
p.atten_table_tot.head()

atten_coef  \
sample mass     radius length density  element energy                
std    0.107074 0.65   0.15   0.537795 Mg      0.84300    0.005930   
                                       Al      1.77880    0.004085   
                                       Ca      3.08450    0.003019   
                                       Cl      1.64259    0.004256   
                                       Mn      1.81070    0.004045   

                                                        self_atten_coef  
sample mass     radius length density  element energy                    
std    0.107074 0.65   0.15   0.537795 Mg      0.84300         0.997041  
                                       Al      1.77880         0.997960  
                                       Ca      3.08450         0.998492  
                                       Cl      1.64259         0.997875  
                                       Mn      1.81070         0.997980

In [110]:
# ratio of self_atten_coef
p.atten_ratio(std_val='std', value_name='values').head()

values  \
                                                       self_atten_coef   
sample mass     radius length density  element energy                    
std    0.107074 0.65   0.15   0.537795 Mg      0.84300        0.997041   
                                       Al      1.77880        0.997960   
                                       Ca      3.08450        0.998492   
                                       Cl      1.64259        0.997875   
                                       Mn      1.81070        0.997980   

                                                            std/sample  \
                                                       self_atten_coef   
sample mass     radius length density  element energy                    
std    0.107074 0.65   0.15   0.537795 Mg      0.84300             1.0   
                                       Al      1.77880             1.0   
                                       Ca      3.08450             1.0   
                                       Cl      1.64259             1.0   
                                       Mn      1.81070             1.0   

                                                                uncert  
                                                       self_atten_coef  
sample mass     radius length density  element energy                   
std    0.107074 0.65   0.15   0.537795 Mg      0.84300             0.0  
                                       Al      1.77880             0.0  
                                       Ca      3.08450             0.0  
                                       Cl      1.64259             0.0  
                                       Mn      1.81070             0.0

# write to excel file

In [111]:
with pd.ExcelWriter('./tmp.xlsx') as f:
    S.mass_table.to_excel(f, sheet_name='mass')
    S.sigma_table.to_excel(f, sheet_name='sigma_table')
    S.sigma_table_tot.to_excel(f, sheet_name='sigma_table_tot')
#    S.shielding_table.to_excel(f, sheet_name='shielding')
    S.shielding_ratio(value_name='values').to_excel(f, sheet_name='shielding_ratio')
    
    p.gamma_table.to_excel(f, sheet_name='gamma')
    p.atten_table.to_excel(f, sheet_name='photon_atten')
#     p.atten_table_tot.to_excel(f, sheet_name='photon_atten_tot')
    p.atten_ratio(value_name='values').to_excel(f, sheet_name='photon_ratio')



# Differences between user table and `periodictable` package

In [112]:
import neutron_analysis.shielding as shielding
import periodictable
import pandas as pd
import numpy as np

In [113]:
df_sigma = shielding.get_default_sigma_table()

In [114]:
df_sigma

,atomic_mass,sigma_a,sigma_s
element,,,
H,1.007940,0.332600,82.02
D,2.014102,0.000519,7.64
He,4.002602,0.007470,1.34
Li,6.941000,70.500000,1.37
Be,9.012182,0.007600,7.63
...,...,...,...
Ra,NaN,12.800000,13.00
Ac,NaN,NaN,NaN
Th,232.038100,7.370000,13.36


In [116]:
# check agains periodic table
df_check = (
    df_sigma
    .reset_index()
    .assign(
        atomic_mass2=lambda df: df['element'].apply(lambda x: getattr(periodictable, x).mass),
        sigma_a2=lambda df: df['element'].apply(lambda x: getattr(periodictable, x).neutron.absorption),
        sigma_s2=lambda df: df['element'].apply(lambda x: getattr(periodictable, x).neutron.total),
    )
    .assign(
        dmass=lambda x: np.abs(x['atomic_mass'] - x['atomic_mass2']),
        da=lambda x: np.abs(x['sigma_a'] - x['sigma_a2']),
        ds=lambda x: np.abs(x['sigma_s'] - x['sigma_s2']),
    )
    .set_index('element')
    
)

In [117]:
# differences in atomic mass
df_check[['atomic_mass','atomic_mass2','dmass']].dropna().query('dmass > 0')

,atomic_mass,atomic_mass2,dmass
element,,,
D,2.014102,2.014102,2.000000e-09
C,12.011000,12.010700,3.000000e-04
N,14.006740,14.006700,4.000000e-05
Na,22.989768,22.989770,2.000000e-06
Al,26.981539,26.981538,1.000000e-06
P,30.973762,30.973761,1.000000e-06
S,32.066000,32.065000,1.000000e-03
Cl,35.452700,35.453000,3.000000e-04
Ti,47.880000,47.867000,1.300000e-02


In [118]:
# differences in sigma_a
df_check[['sigma_a','sigma_a2','da']].dropna().query('da > 0')

,sigma_a,sigma_a2,da
element,,,
Nd,51.0,50.5,0.5


In [119]:
# differences in sigma_s
df_check[['sigma_s','sigma_s2','ds']].dropna().query('ds > 0')

,sigma_s,sigma_s2,ds
element,,,
Sm,39.0,39.400,0.400
U,8.9,8.908,0.008
